In [66]:
import numpy as np
import pickle
from pyfasttext import FastText
from collections import defaultdict

In [4]:
model = FastText("/data/rali7/Tmp/solimanz/data/wikipedia/wiki.en.bin")

In [55]:
with open("/data/rali7/Tmp/solimanz/data/datasets/skills/skills550.pkl", "rb") as f:
    skills550 = pickle.load(f)
with open("/data/rali7/Tmp/solimanz/data/datasets/skills/skills7000.pkl", "rb") as f:
    skills7k = pickle.load(f)

In [56]:
s5 = set()
for v in skills550.values():
    for elem in v:
        s5.add(elem)

In [57]:
s7 = set()
for v in skills7k.values():
    for elem in v:
        s7.add(elem)

In [58]:
def get_skills_emb(id_skills):
    skills_emb = dict()
    for id_, skills in id_skills.items():
        skills_emb[id_] = model.get_numpy_sentence_vector(" ".join(skills))
    
    return skills_emb

In [61]:
skills = s5.union(s7)
skills.add('<NS>')

In [62]:
skills_fasttext = np.zeros((len(skills), 300))

In [63]:
skills_id = {skill: id_ for id_, skill in enumerate(skills)}

In [64]:
for skill, id_ in skills_id.items():
    skills_fasttext[id_] = model.get_numpy_sentence_vector(skill)

In [65]:
np.save('/data/rali7/Tmp/solimanz/data/datasets/skills/skills_embs.npy', skills_fasttext)

In [69]:
def get_user_to_skill_id(id_skills, skills_id):
    user_skills = defaultdict(list)
    for id_, skills in id_skills.items():
        for skill in skills:
            user_skills[id_].append(skills_id[skill])
    return user_skills

In [70]:
user_skills_550 = get_user_to_skill_id(skills550, skills_id)
user_skills_7k = get_user_to_skill_id(skills7k, skills_id)

In [71]:
with open('/data/rali7/Tmp/solimanz/data/datasets/skills/user_skills_550.pkl', 'wb') as f:
    pickle.dump(user_skills_550, f)
with open('/data/rali7/Tmp/solimanz/data/datasets/skills/user_skills_7k.pkl', 'wb') as f:
    pickle.dump(user_skills_7k, f)

In [73]:
with open('/data/rali7/Tmp/solimanz/data/datasets/skills/skills_id.pkl', 'wb') as f:
    pickle.dump(skills_id, f)